In [21]:
import tensorflow as tf

batch_size = 16
img_size = (224, 224)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",  # Parent directory of your dataset
    validation_split=0.1,  # 20% of data for validation
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="binary",  # For binary classification (labels will be 0 or 1)
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    validation_split=0.1,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="binary",
)

Found 84 files belonging to 2 classes.
Using 76 files for training.
Found 84 files belonging to 2 classes.
Using 8 files for validation.


In [22]:
from tensorflow.keras import layers, models

# Load MobileNet with pretrained weights.
base_model = tf.keras.applications.MobileNet(
    input_shape=(224, 224, 3),
    include_top=False,  # Remove final classification layers
    weights="imagenet",
    pooling="avg",  # Global average pooling gives a fixed-length vector output
)
base_model.trainable = False  # Freeze the base model

# Create the new model by stacking a classification head on top of MobileNet.
inputs = tf.keras.Input(shape=(224, 224, 3))
# Preprocess inputs using MobileNet's own preprocessing function
x = tf.keras.applications.mobilenet.preprocess_input(inputs)
# Extract features using MobileNet
x = base_model(x, training=False)
# Add a Dense layer for binary classification (sigmoid activation outputs a probability)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = models.Model(inputs, outputs)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_3 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract_3 (Subtract)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenet_1.00_224 (Functional) │ (None, 1024)           │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,229,889 (12.32 MB)

 Trainable params: 1,025 (4.00 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [23]:
epochs = 10
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 218ms/step - accuracy: 0.4836 - loss: 0.9062 - val_accuracy: 0.6250 - val_loss: 0.7571
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.6587 - loss: 0.6326 - val_accuracy: 0.5000 - val_loss: 0.6742
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.7292 - loss: 0.5392 - val_accuracy: 0.3750 - val_loss: 0.6308
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.8276 - loss: 0.4347 - val_accuracy: 0.7500 - val_loss: 0.5117
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.8842 - loss: 0.3255 - val_accuracy: 0.8750 - val_loss: 0.4204
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.8729 - loss: 0.3388 - val_accuracy: 0.8750 - val_loss: 0.3752
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.9199 - loss: 0.3061 - val_accuracy: 0.8750 - val_loss: 0.3564
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - accuracy: 0.9278 - loss: 0.2311 - val_accuracy: 0.8750 - val_loss:

In [24]:
# Evaluate the model
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.8750 - loss: 0.3052
Validation Accuracy: 87.50%


In [25]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf


def load_and_preprocess_image(img_path):
    try:
        # Load the image using PIL
        with Image.open(img_path) as img:
            # Resize the image
            img = img.resize((224, 224))
            # Convert to RGB if necessary
            if img.mode != "RGB":
                img = img.convert("RGB")
            # Convert to numpy array
            img_array = np.array(img)
            # Add batch dimension
            img_array = np.expand_dims(img_array, axis=0)
            return img_array
    except Exception as e:
        print(f"Error loading image {img_path}: {str(e)}")
        return None


# Print class names to see the mapping
print("Class names:", train_ds.class_names)

# List your sample image paths
sample_images = [
    "sample_1.png",
    "sample_2.png",
    "sample_3.jpeg",
    "sample_4.png",
    "sample_5.png",
    "sample_6.jpeg",
]

for img_path in sample_images:
    # Preprocess the image
    img_input = load_and_preprocess_image(img_path)
    if img_input is not None:
        # Run inference on the image
        prediction = model.predict(img_input, verbose=0)
        # With the default ordering, if prediction < 0.5, predicted label is 0 ("Ghibli"), else 1 ("Non_Ghibli")
        predicted_label = 0 if prediction[0][0] < 0.5 else 1
        predicted_class = train_ds.class_names[predicted_label]
        confidence = prediction[0][0] if predicted_label == 1 else 1 - prediction[0][0]
        print(
            f"Prediction for {img_path}: {predicted_class} (confidence: {confidence:.2%})"
        )

Class names: ['Ghibli', 'Non_Ghibli']
Prediction for sample_1.png: Ghibli (confidence: 82.03%)
Prediction for sample_2.png: Non_Ghibli (confidence: 58.90%)
Prediction for sample_3.jpeg: Ghibli (confidence: 60.72%)
Prediction for sample_4.png: Ghibli (confidence: 96.27%)
Prediction for sample_5.png: Ghibli (confidence: 73.52%)
Prediction for sample_6.jpeg: Ghibli (confidence: 97.20%)


In [26]:
model.save("model.keras")